In [ ]:
pip install torchaudio

In [ ]:
pip install git+https://github.com/ContinualAI/avalanche.git

In [1]:
import os
import torchaudio
from torch import Tensor
from torch.nn import Module
from torch.utils.data import Dataset
from torchvision.datasets.utils import download_url
from torchaudio.datasets.utils import extract_archive
import librosa
import numpy
from typing import Tuple
import numpy as np
import torch
from matplotlib import pyplot as plt
from torch.utils.data.dataloader import DataLoader
from avalanche.benchmarks import NCScenario, nc_benchmark, ni_benchmark

In [14]:
def add_white_noise(x, rate=0.002):
        return x + rate*np.random.randn(len(x))

    # data augmentation: shift sound in timeframe
def shift_sound(x, rate=2):
    return np.roll(x, int(len(x)//rate))

    # data augmentation: stretch sound
def stretch_sound(x, rate=1.1):
    input_length = len(x)
    x = librosa.effects.time_stretch(x, rate)
    if len(x)>input_length:
        return x[:input_length]

**CREAZIONE DATASET - ESC50 RITORNA L'AUDIO COME UN ARRAY - ES50_V2 RITORNA L'AUDIO COMEUN IMMAGINE DA POTER PASSARE AD UNA CNN**

In [15]:

# Download URL and checksums
URL = {
    "esc-10": "https://github.com/karoldvl/ESC-50/archive/master.zip",
    "esc-50": "https://github.com/karoldvl/ESC-50/archive/master.zip",
    # "esc-us": None,
}

_CHECKSUMS = {
    "esc-10": None,
    "esc-50": None,
    # "esc-us": None,
}

# Constant
ARCHIVE_BASENAME = "ESC-50-master"
FOLDER_IN_ARCHIVE = "ESC-50-master"
AUDIO_FOLDER = "audio"
META_FOLDER = "meta"
AVAILABLE_VERSION = list(URL.keys())

# Default parameters
FOLDS = (1, 2, 3, 4, 5)


class ESC50(Dataset):
    """
    ESC datasets
    Args:
        root (string): Root directory of datasets where directory
            ``ESC-50-master`` exists or will be saved to if download is set to True.
        download (bool, optional): If true, download the dataset from the internet
            and puts it in root directory. If datasets is already downloaded, it is
            not downloaded again.
    """
    NB_CLASS = 50

    def __init__(self,
                 root: str,
                 train: bool = True,
                 folds: tuple = FOLDS,
                  data_aug=False,
                 download: bool = False,
                 transform: Module = None) -> None:

        super().__init__()
        self.train = train
        self.root = root
        self.required_folds = folds
        self.transform = transform
        self.data_aug = data_aug
        self.url = URL["esc-50"]
        self.nb_class = 50
        self.target_directory = os.path.join(self.root, FOLDER_IN_ARCHIVE)

        # Dataset must exist to continue
        if download:
            print('start download')
            self.download()
        # elif not self.check_integrity(self.target_directory):
        #     raise RuntimeError("Dataset not found or corrupted. \n\
        #         You can use download=True to download it.")

        # Prepare the medata
      
        self._filenames = []
        self._folds = []
        self.targets = []
        self._esc10s = []
        self._load_metadata()
    
    def add_white_noise(x, rate=0.002):
        return x + rate*np.random.randn(len(x))

    # data augmentation: shift sound in timeframe
    def shift_sound(x, rate=2):
        return np.roll(x, int(len(x)//rate))

    # data augmentation: stretch sound
    def stretch_sound(x, rate=1.1):
        input_length = len(x)
        x = librosa.effects.time_stretch(x, rate)
        if len(x)>input_length:
            return x[:input_length]
    def __getitem__(self, index: int) -> Tuple[Tensor, int]:
        """
        Args:
            index (int): Index
        Returns:
            tuple: (raw_audio, sr, target).
        """
        
        
        data, target = self.load_item(index)
        # augumentations in wave domain.
        
        if self.data_aug:
            r = np.random.rand()
            
            if r < 0.3:
                
                data = add_white_noise(data)
           
            r = np.random.rand()
            if r < 0.3:
               
                data = shift_sound(data, rate=1+np.random.rand())
            
            #r = np.random.rand()
            #if r < 0.3:
                #data = stretch_sound(data, rate=0.8+np.random.rand()*0.4)
        
        if self.transform is not None:
            data = self.transform(data)
            data = data.squeeze()
        
        return np.float32(data), target

    def __len__(self) -> int:
        return len(self._filenames)

    def _load_metadata(self) -> None:
        
        """Read the metadata csv file and gather the information needed."""
        # HEADER COLUMN NUMBER
        c_filename = 0
        c_fold = 1
        c_target = 2
        c_esc10 = 4

        # Read the csv file and remove header
        path = os.path.join(self.target_directory, META_FOLDER, "esc50.csv")
        
        with open(path, "r") as fp:
            data = fp.read().splitlines()[1:]
            
            for line in data:
                items = line.split(",")
                if self.train:
                  if int(items[c_fold]) < 5:
                    self._filenames.append(items[c_filename])
                    self._folds.append(int(items[c_fold]))
                    self.targets.append(int(items[c_target]))
                    self._esc10s.append(eval(items[c_esc10]))
                else:
                  if int(items[c_fold]) == 5:
                    self._filenames.append(items[c_filename])
                    self._folds.append(int(items[c_fold]))
                    self.targets.append(int(items[c_target]))
                    self._esc10s.append(eval(items[c_esc10]))

        self._filenames = numpy.asarray(self._filenames)
        self._folds = numpy.asarray(self._folds)
        self.targets = numpy.asarray(self.targets)
        self._esc10s = numpy.asarray(self._esc10s)

        # Keep only the required folds
        

        
        folds_mask = sum([self._folds == f for f in self.required_folds]) >= 1
        
        self._filenames = self._filenames[folds_mask]
        self.targets = self.targets[folds_mask]
        self._esc10s = self._esc10s[folds_mask]

    def download(self) -> None:
        
        """Download the dataset and extract the archive"""
        if self.check_integrity(self.target_directory):
            
            print("Dataset already downloaded and verified.")

        else:
           
            archive_path = os.path.join(self.root, FOLDER_IN_ARCHIVE + ".zip")
           
            download_url(self.url, self.root, filename='ESC-50-master.zip')
            
            extract_archive(archive_path, self.root)
    # data augmentation: add white noise
    
    def check_integrity(self, path, checksum=None) -> bool:
        
        """Check if the dataset already exist and if yes, if it is not corrupted.
        Returns:
            bool: False if the dataset doesn't exist or if it is corrupted.
        """
        if not os.path.isdir(path):
            return False

        # TODO add checksum verification
        return True

    def load_item(self, index: int) -> Tuple[Tensor, int]:
        
        filename = self._filenames[index]
        target = self.targets[index]

        path = os.path.join(self.target_directory, AUDIO_FOLDER, filename)
        waveform, sample_rate = librosa.load(path, sr=44100)
        
        # waveform, sample_rate = torchaudio.load(path)
        
        return waveform, target


class ESC50_v2(Dataset):
    """
    ESC datasets
    Args:
        root (string): Root directory of datasets where directory
            ``ESC-50-master`` exists or will be saved to if download is set to True.
        download (bool, optional): If true, download the dataset from the internet
            and puts it in root directory. If datasets is already downloaded, it is
            not downloaded again.
    """
    NB_CLASS = 50

    def __init__(self,
                 root: str,
                 train: bool = True,
                 folds: tuple = FOLDS,
                  data_aug=False,
                 download: bool = False,
                 transform: Module = None) -> None:

        super().__init__()
        self.train = train
        self.root = root
        self.required_folds = folds
        self.transform = transform
        self.data_aug = data_aug
        self.url = URL["esc-50"]
        self.nb_class = 50
        self.target_directory = os.path.join(self.root, FOLDER_IN_ARCHIVE)

        # Dataset must exist to continue
        if download:
            print('start download')
            self.download()
        # elif not self.check_integrity(self.target_directory):
        #     raise RuntimeError("Dataset not found or corrupted. \n\
        #         You can use download=True to download it.")

        # Prepare the medata
        
        self._filenames = []
        self._folds = []
        self.targets = []
        self._esc10s = []
        self._load_metadata()
    
    def add_white_noise(x, rate=0.002):
        return x + rate*np.random.randn(len(x))

    # data augmentation: shift sound in timeframe
    def shift_sound(x, rate=2):
        return np.roll(x, int(len(x)//rate))

    # data augmentation: stretch sound
    def stretch_sound(x, rate=1.1):
        input_length = len(x)
        x = librosa.effects.time_stretch(x, rate)
        if len(x)>input_length:
            return x[:input_length]
    def __getitem__(self, index: int) -> Tuple[Tensor, int]:
        """
        Args:
            index (int): Index
        Returns:
            tuple: (raw_audio, sr, target).
        """
        
        
        data, target = self.load_item(index)
        # augumentations in wave domain.
        
        if self.data_aug:
            r = np.random.rand()
            
            if r < 0.3:
                
                data = add_white_noise(data)
           
            r = np.random.rand()
            if r < 0.3:
               
                data = shift_sound(data, rate=1+np.random.rand())
            
            #r = np.random.rand()
            #if r < 0.3:
                #data = stretch_sound(data, rate=0.8+np.random.rand()*0.4)
        
        if self.transform is not None:
            data = self.transform(data)
            data = data.squeeze()
        
        return np.float32(data), target

    def __len__(self) -> int:
        return len(self._filenames)

    def _load_metadata(self) -> None:
        
        """Read the metadata csv file and gather the information needed."""
        # HEADER COLUMN NUMBER
        c_filename = 0
        c_fold = 1
        c_target = 2
        c_esc10 = 4

        # Read the csv file and remove header
        path = os.path.join(self.target_directory, META_FOLDER, "esc50.csv")
        
        with open(path, "r") as fp:
            data = fp.read().splitlines()[1:]
            
            for line in data:
                items = line.split(",")
                if self.train:
                  if int(items[c_fold]) < 5:
                    self._filenames.append(items[c_filename])
                    self._folds.append(int(items[c_fold]))
                    self.targets.append(int(items[c_target]))
                    self._esc10s.append(eval(items[c_esc10]))
                else:
                  if int(items[c_fold]) == 5:
                    self._filenames.append(items[c_filename])
                    self._folds.append(int(items[c_fold]))
                    self.targets.append(int(items[c_target]))
                    self._esc10s.append(eval(items[c_esc10]))

        self._filenames = numpy.asarray(self._filenames)
        self._folds = numpy.asarray(self._folds)
        self.targets = numpy.asarray(self.targets)
        self._esc10s = numpy.asarray(self._esc10s)

        # Keep only the required folds
        

        
        folds_mask = sum([self._folds == f for f in self.required_folds]) >= 1
        
        self._filenames = self._filenames[folds_mask]
        self.targets = self.targets[folds_mask]
        self._esc10s = self._esc10s[folds_mask]
        self.resample = torchaudio.transforms.Resample(
            orig_freq=44100, new_freq=8000
        )
        self.melspec = torchaudio.transforms.MelSpectrogram(
            sample_rate=8000)
        self.db = torchaudio.transforms.AmplitudeToDB(top_db=80)

    def download(self) -> None:
        
        """Download the dataset and extract the archive"""
        if self.check_integrity(self.target_directory):
            
            print("Dataset already downloaded and verified.")

        else:
            
            archive_path = os.path.join(self.root, FOLDER_IN_ARCHIVE + ".zip")
            
            download_url(self.url, self.root, filename='ESC-50-master.zip')
            
            extract_archive(archive_path, self.root)
    # data augmentation: add white noise
    
    def check_integrity(self, path, checksum=None) -> bool:
        
        """Check if the dataset already exist and if yes, if it is not corrupted.
        Returns:
            bool: False if the dataset doesn't exist or if it is corrupted.
        """
        if not os.path.isdir(path):
            return False

        # TODO add checksum verification
        return True

    def load_item(self, index: int) -> Tuple[Tensor, int]:
        
        filename = self._filenames[index]
        target = self.targets[index]

        path = os.path.join(self.target_directory, AUDIO_FOLDER, filename)
        wav, _ = torchaudio.load(path)
        xb = self.db(
            self.melspec(
                self.resample(wav)
            )
        )
        
        # waveform, sample_rate = torchaudio.load(path)
        
        return xb, target

In [16]:
train_dataset = ESC50_v2(root= os.path.abspath(os.getcwd()),download =True, data_aug=True, train=True)
test_dataset = ESC50_v2(root= os.path.abspath(os.getcwd()),download =True, data_aug=False, train=False)

'''
train_dataset = ESC50(root= os.path.abspath(os.getcwd()),download =True, data_aug=False, train=True)
test_dataset = ESC50(root= os.path.abspath(os.getcwd()),download =True, data_aug=False, train=False)
'''

start download
Dataset already downloaded and verified.
start download
Dataset already downloaded and verified.


'\ntrain_dataset = ESC50(root= os.path.abspath(os.getcwd()),download =True, data_aug=False, train=True)\ntest_dataset = ESC50(root= os.path.abspath(os.getcwd()),download =True, data_aug=False, train=False)\n'

In [4]:
train_dataset[0][0].shape

(1, 128, 201)

**DEFINIZIONE SCENARI**

In [36]:
scenario = nc_benchmark(
            train_dataset=train_dataset,
            test_dataset=test_dataset,
            n_experiences=10,
            task_labels=True,
            seed=123,
            fixed_class_order=None,
            shuffle=True,
            class_ids_from_zero_in_each_exp=True)


scenario2 = ni_benchmark(
            train_dataset, test_dataset, 5, shuffle=True, seed=1234,
            balance_experiences=True)

**FUNZIONE CHE FA IL PLOT DELL'AUDIO NEL CASO IN CUI VIENE GENERATO DA UNA LISTA (ESC-50)**

In [ ]:
def show_wave(x):
    plt.plot(x)
    plt.show()
    
def check_audio_benchmark(benchmark_instance):
  print('The benchmark instance contains',
  len(benchmark_instance.train_stream), 'training experiences.')

  for i, exp in enumerate(benchmark_instance.train_stream):
      dataset, t = exp.dataset, exp.task_label
    

      dl = DataLoader(dataset, batch_size=300)
      print('Train experience', exp.current_experience)
      for x, y, z in dl.dataset:
        show_wave(x)
        break
check_audio_benchmark(scenario)

**MOSTRIAMO DELLE CARATTERISTICHE DELLO SCENARIO**

In [32]:
train_stream = scenario.train_stream
print(train_stream.name)

# we have access to the scenario from which the stream was taken
train_stream.benchmark

# we can slice and reorder the stream as we like!
substream = train_stream[0]
substream = train_stream[0:2]
substream = train_stream[0,2,1]

print(len(substream))
train_stream = scenario.train_stream
test_stream = scenario.test_stream

# iterating over the train stream
for experience in train_stream:
  print("Start of task ", experience.task_label)
  print('Classes in this task:', experience.classes_in_this_experience)
  
  # The current Pytorch training set can be easily recovered through the
  # experience
  current_training_set = experience.dataset
  # ...as well as the task_label
  print('Task {}'.format(experience.task_label))
  print('This task contains', len(current_training_set), 'training examples')

  # we can recover the corresponding test experience in the test stream
  current_test_set = test_stream[experience.current_experience].dataset
  print('This task contains', len(current_test_set), 'test examples')

train
3
Start of task  0
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 0
This task contains 320 training examples
This task contains 80 test examples
Start of task  1
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 1
This task contains 320 training examples
This task contains 80 test examples
Start of task  2
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 2
This task contains 320 training examples
This task contains 80 test examples
Start of task  3
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 3
This task contains 320 training examples
This task contains 80 test examples
Start of task  4
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 4
This task contains 320 training examples
This task contains 80 test examples


**DEFINIZIONE DI MODELLI DI RETI NEURALI**

1.   Audio Netowrk
2.   CNN_2 (riadattamento di quella di Avalanche)



In [7]:
import torch.nn as nn
import torchaudio.transforms as T
import random
class AudioNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        filt_size = (3, 3)
        pool_size = (2, 2)
        num_classes=10
        
        
        self.use_bias = True
        self.b_norm2d = nn.BatchNorm2d(1)
        self.mlp = nn.Sequential(
            
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128,num_classes),
            nn.Softmax(dim=1)

        )

        self.pool = nn.Sequential(
            nn.MaxPool2d((16,24)),
            nn.Flatten()           
            
        )
        self.conv = nn.Sequential(
            nn.BatchNorm2d(1),
            #block1
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=filt_size, bias=self.use_bias, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=filt_size, bias=self.use_bias, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(pool_size, stride=2),

            #block2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=filt_size, bias=self.use_bias,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=filt_size, bias=self.use_bias, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(pool_size, stride=2),

            #block3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=filt_size, bias=self.use_bias, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=filt_size, bias=self.use_bias, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(pool_size, stride=2),

            #block4
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=filt_size, bias=self.use_bias, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=filt_size, bias=self.use_bias, padding=1),
        )
    
        

    def forward(self, X: torch.Tensor):
     
          
          inp = self.b_norm2d(X)
          output = self.conv(inp) #512 values
          output = self.pool(output)
            #MLP 
          output = self.mlp(output)

            
          
          return output


# SIMPLE_CNN2: presa da Avalcnhe ed adatattata al formato audio

import torch.nn as nn

from avalanche.models.dynamic_modules import MultiTaskModule, \
    MultiHeadClassifier


class SimpleCNN_2(nn.Module):

    def __init__(self, num_classes=10):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=0),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=0),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25),
            nn.Conv2d(64, 64, kernel_size=1, padding=0),
            nn.ReLU(inplace=True),
            nn.AdaptiveMaxPool2d(1),
            nn.Dropout(p=0.25)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x



In [8]:
class Net(nn.Module):
    def __init__(self, n_input=1, n_output=7, stride=16, n_channel=32):
        super(Net, self).__init__()
        
        self.nn = nn.Sequential(
            #Fully connected layeres and Dropout
            nn.Linear(in_features=40, out_features=256),
            nn.ReLU(),
            nn.Dropout2d(0.25),
            nn.Linear(in_features=256, out_features=512),
            nn.ReLU(),
            nn.Dropout2d(0.25),
            nn.Linear(in_features=512, out_features=n_channel),
        )
        
        #self.linear = F.log_softmax(nn.Linear(n_channel, n_output), dim=1)
        
        self.linear = nn.Linear(n_channel, n_output)

    def forward(self, x):
        embedding = self.nn(x)
        x = F.log_softmax(self.linear(embedding), dim=1)
        return embedding, x

**DEFINIZIONI DI MODELLI, OPTIMIZER E CRITERION**

In [26]:
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from avalanche.models import SimpleMLP, MTSimpleMLP, SimpleCNN

model = SimpleMLP(input_size=128*201)
model2 = AudioNetwork()
model4 = SimpleCNN_2()
model3 = MTSimpleMLP(input_size=220500)
#model5 = AudioNet()
model6 = Net()
#optimizer = Adam(model.parameters(), lr=0.01)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()





**DEFINIZIONI STRATEGIE**

In [33]:

from avalanche.training.strategies import EWC, Replay, Naive
cl_strategy = EWC(
    model4, optimizer, criterion, ewc_lambda=0.4,
    train_mb_size=4, train_epochs=5, eval_mb_size=1
)


cl_strategy2 = Replay( model4, optimizer, criterion,
    train_mb_size=15, train_epochs=10, eval_mb_size=15)

#cl_strategy3 = Replay( model5, optimizer, criterion,
#    train_mb_size=5, train_epochs=10, eval_mb_size=5)

cl_strategy4 = Replay( model2, optimizer, criterion,
    train_mb_size=5, train_epochs=10, eval_mb_size=5)

cl_strategy5 = Naive(model4, optimizer, criterion,
    train_mb_size=32, train_epochs=15, eval_mb_size=32)


In [34]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy2.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(scenario.test_stream))
    #print(cl_strategy.eval(scenario.test_stream))
    print('**************************************')

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████| 22/22 [00:31<00:00,  1.43s/it]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.5765
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2625
100%|██████████| 22/22 [00:31<00:00,  1.42s/it]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.1669
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2469
100%|██████████| 22/22 [00:31<00:00,  1.44s/it]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.0357
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3000
100%|██████████| 22/22 [00:31<00:00,  1.45s/it]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.9205
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3125
100%|██████████| 22/22 [00:31<00:00,  1.43s/it]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/T

In [54]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

print('Starting experiment...')
results_2 = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy3.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results_2.append(cl_strategy.eval(scenario.test_stream))
    #print(cl_strategy.eval(scenario.test_stream))
    print('**************************************')

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████| 64/64 [10:44<00:00, 10.08s/it]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.0149
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1719
100%|██████████| 64/64 [01:06<00:00,  1.03s/it]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.0692
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3000
100%|██████████| 64/64 [01:06<00:00,  1.04s/it]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.8623
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3438
100%|██████████| 64/64 [01:09<00:00,  1.09s/it]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.7790
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3656
100%|██████████| 64/64 [01:11<00:00,  1.11s/it]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/T

In [37]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

print('Starting experiment...')
results3 = []
for experience in scenario2.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy2.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results3.append(cl_strategy.eval(scenario2.test_stream))
    #print(cl_strategy.eval(scenario.test_stream))
    print('**************************************')

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
0it [00:00, ?it/s]

IndexError: ignored

**Confronto con Permuted Mnist**

In [19]:
from avalanche.benchmarks.classic import CORe50, SplitTinyImageNet, \
SplitCIFAR10, SplitCIFAR100, SplitCIFAR110, SplitMNIST, RotatedMNIST, \
PermutedMNIST, SplitCUB200, SplitImageNet

# creating PermutedMNIST (Task-Incremental)
perm_mnist = PermutedMNIST(
    n_experiences=2,
    seed=1234,
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [102]:
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

# iterating over the train stream
for experience in train_stream:
  print("Start of task ", experience.task_label)
  print('Classes in this task:', experience.classes_in_this_experience)
  
  # The current Pytorch training set can be easily recovered through the
  # experience
  current_training_set = experience.dataset
  # ...as well as the task_label
  print('Task {}'.format(experience.task_label))
  print('This task contains', len(current_training_set), 'training examples')

  # we can recover the corresponding test experience in the test stream
  current_test_set = test_stream[experience.current_experience].dataset
  print('This task contains', len(current_test_set), 'test examples')

Start of task  0
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 0
This task contains 60000 training examples
This task contains 10000 test examples
Start of task  1
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 1
This task contains 60000 training examples
This task contains 10000 test examples


In [ ]:
from avalanche.benchmarks.classic import SplitMNIST, core50

#benchmark = CORe50(scenario="nc")

# scenario
scenario2 = SplitMNIST(n_experiences=5, seed=1)

In [ ]:
class MyStrategy():
    """My Basic Strategy"""

    def __init__(self, model, optimizer, criterion):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion

    def train(self, experience):
        # here you can implement your own training loop for each experience (i.e. 
        # batch or task).

        train_dataset = experience.dataset
        t = experience.task_label
        train_data_loader = DataLoader(
            train_dataset, num_workers=4, batch_size=128
        )

        for epoch in range(1):
            for mb in train_data_loader:
                # you magin here...
                pass

    def test(self, experience):
        # here you can implement your own test loop for each experience (i.e. 
        # batch or task).

        test_dataset = experience.dataset
        t = experience.task_label
        test_data_loader = DataLoader(
            test_dataset, num_workers=4, batch_size=128
        )

        # test here

In [21]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.models import SimpleMLP
from avalanche.training.strategies import EWC

model = SimpleMLP(num_classes=scenario.n_classes)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

cl_strategy = MyStrategy(model, optimizer, criterion)


NameError: ignored

In [ ]:
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    #results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training completed
Computing accuracy on the whole test set
Start of experience:  1
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Training completed
Computing accuracy on the whole test set
Start of experience:  2
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Training completed
Computing accuracy on the whole test set
Start of experience:  3
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Training completed
Computing accuracy on the whole test set
Start of experience:  4
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Training completed
Computing accuracy on the whole test set


In [22]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.models import SimpleMLP
from avalanche.training.strategies import EWC


model = SimpleMLP()
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()
cl_strategy = EWC(
    model, optimizer, criterion, ewc_lambda=0.4,
    train_mb_size=100, train_epochs=4, eval_mb_size=100
)


In [24]:
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy.train()
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


TypeError: ignored